# *** camb15 generated datasets for TT_full, TE_full and EE_full components ***

In [ ]:
## install
#!pip install camb

# import libraries

In [ ]:
## import libraries
%matplotlib inline
import sys, platform, os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
import camb
from time import time
from camb import model, initialpower

print('CAMB version: %s '%camb.__version__)
print('Using CAMB installed at %s'%(os.path.realpath(os.path.join(os.getcwd(),'..'))))

import warnings
warnings.filterwarnings("ignore")

In [ ]:
## drive
#from google.colab import drive
#drive.mount('/content/drive/')

In [ ]:
## get ns and As def.s
ns_As   = []
mean_ns = 0.96
mean_As = 3.09
std     = 0.3
np.random.seed(189921)

while (len(ns_As)<=10000):
    ns = np.random.uniform(0.55, 1.37)
    As = np.random.uniform(2.68, 3.5)
    if ns>0.1 and ns<1.82 and As<5.09 and As>1.09:
        ns_As.append([ns,As])

ns_As = np.array(ns_As)

In [ ]:
## add parameters from camb with unlensed_scalar
ti = time()

TT = []
EE = []
TE = []

for i in range(300, len(ns_As)):
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=0.6732117E+02, ombh2=0.2238280E-01, omch2=0.1201075E+00, mnu=0.06, omk=0, tau=0.5430842E-01);
    pars.InitPower.set_params(ns=ns_As[i,0], As=np.exp(ns_As[i,1])*1e-10, r=0);
    pars.set_for_lmax(lmax=2508, lens_potential_accuracy=0);
    results = camb.get_results(pars);
    powers  = results.get_cmb_power_spectra(pars, CMB_unit='muK', 
                                            lmax=2508, spectra=['unlensed_scalar']);
    TT.append(powers['unlensed_scalar'][2:,0])
    EE.append(powers['unlensed_scalar'][2:1997,1])
    TE.append(powers['unlensed_scalar'][2:1997,3])
    if i%100==0: 
        print(i, 'elapsed time: %.3f h'%((time()-ti)/60/60))

In [ ]:
## get dataset files of ns and As for TT, TE, EE.
df_TT = pd.DataFrame()
df_TE = pd.DataFrame()
df_EE = pd.DataFrame()

for df in [df_TT, df_TE, df_EE]:
    df['ns'] = ns_As[:,0]
    df['As'] = ns_As[:,1]
    
df_TT = pd.concat((df_TT, pd.DataFrame(TT,columns=list(map(str,range(2,2509))))),axis=1)
df_TE = pd.concat((df_TE, pd.DataFrame(TE,columns=list(map(str,range(2,1997))))),axis=1)
df_EE = pd.concat((df_EE, pd.DataFrame(EE,columns=list(map(str,range(2,1997))))),axis=1)

df_TT.to_csv('TT_full.csv', index=False)
df_TE.to_csv('TE_full.csv', index=False)
df_EE.to_csv('EE_full.csv', index=False)